- 수집할 페이지의 주소
- https://pjt3591oo.github.io


In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
from IPython.display import clear_output
import os


In [2]:
# 요청하는 함수
def getSource(site):
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
        
    }
    
    # 요청한다.
    response = requests.get(site, headers = header_info)
#     print(response.text)

    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    return soup


In [3]:
getSource('https://pjt3591oo.github.io')

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Home</title>
<meta content="멍개의 개발 블로그입니다. 궁금하신 사항 혹은 전달하고 싶은 내용이 있으시면 메일로 문의 주세요." name="description"/>
<link href="/assets/main.css" rel="stylesheet"/>
<link href="/" rel="canonical"/>
<link href="/feed.xml" rel="alternate" title="Home" type="application/rss+xml"/>
</head>
<body>
<header class="site-header" role="banner">
<div class="wrapper">
<a class="site-title" href="/">Home</a>
<nav class="site-nav">
<input class="nav-trigger" id="nav-trigger" type="checkbox"/>
<label for="nav-trigger">
<span class="menu-icon">
<svg height="15px" viewbox="0 0 18 15" width="18px">
<path d="M18,1.484c0,0.82-0.665,1.484-1.484,1.484H1.484C0.665,2.969,0,2.304,0,1.484l0,0C0,0.665,0.665,0,1.484,0 h15.031C17.335,0,18,0.665,18,1.484L18,1.484z" fill="#424242"></path>
<path d="M18,7.516C18,8.335,17.335,9,16.516,9H

In [4]:
soup = getSource('https://pjt3591oo.github.io/')
soup


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Home</title>
<meta content="멍개의 개발 블로그입니다. 궁금하신 사항 혹은 전달하고 싶은 내용이 있으시면 메일로 문의 주세요." name="description"/>
<link href="/assets/main.css" rel="stylesheet"/>
<link href="/" rel="canonical"/>
<link href="/feed.xml" rel="alternate" title="Home" type="application/rss+xml"/>
</head>
<body>
<header class="site-header" role="banner">
<div class="wrapper">
<a class="site-title" href="/">Home</a>
<nav class="site-nav">
<input class="nav-trigger" id="nav-trigger" type="checkbox"/>
<label for="nav-trigger">
<span class="menu-icon">
<svg height="15px" viewbox="0 0 18 15" width="18px">
<path d="M18,1.484c0,0.82-0.665,1.484-1.484,1.484H1.484C0.665,2.969,0,2.304,0,1.484l0,0C0,0.665,0.665,0,1.484,0 h15.031C17.335,0,18,0.665,18,1.484L18,1.484z" fill="#424242"></path>
<path d="M18,7.516C18,8.335,17.335,9,16.516,9H

In [13]:
# 

# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(soup) :
#     print(soup)
    # 데이터가 있는 영역 전체를 가져온다.
    a1 = soup.select_one('body > main > div > div')
#     print(a1)
    
# 데이터가 있는 p 태그를 가져온다.
    a2 = a1.select('div.p')
#     print(a2)

    # 데이터를 담을 딕셔너리
    data_dict = {
        '큰제목':[],
        '작은제목':[],
        '날짜':[],
        '작성자':[]
    }
# div 태그의 수 만큼 반복한다.
    for div in a2 :
        # 큰 제목을 가져온다.
        a3 = div.select_one('h3 > a')
        data1 = a3.text.strip()
        # 작은 제목을 가져온다
        a4 = div.select_one('h4')
        data2 = a4.text.strip().split()
        # 만약 길이가 0인 문자열이 추출됬다면 결측치로 변경한다.
        if len(data2) == 0 :
            data2 = np.nan


        # 날짜, 이름을 가져온다.
        a5 = div.select_one('p > span')
        data3 = a5.text.strip()
#         print(data3)
        data4 = data3.split('|')
        data5 = data4[0].strip()
        data6 = data4[1].strip()
        
#         print(data1)
#         print(data2)
#         print(data5)
#         print(data6)
        
        # 딕셔너리에 데이터를 담는다
        data_dict['큰제목'].append(data1)
        data_dict['작은제목'].append(data2)
        data_dict['날짜'].append(data5)
        data_dict['작성자'].append(data6)
        
    df1= pd.DataFrame(data_dict)
#     display(df1)
    if os.path.exists('data1.csv') == False :
        # 파일이 없을 경우
        df1.to_csv('data1.csv', encoding='utf-8-sig', index=False)
    else :
        df1.to_csv('data1.csv',encoding='utf-8-sig' , index=False, header = False, mode='a')

In [14]:
# 다음페이지 존재 여부 확인하는 함수
def getNextPage(soup):
    a1 = soup.select_one('body > main > div > div > div.pagination > ul > li:nth-child(6) > a')
#     print(a1)
    
    # 만약 가져온 것이 있다면..
    if a1 != None :
        # a 태그의 href 속성 값을 가져온다
        href = a1.attrs['href']
#         print(href)
        return href
    else :
        return None
    

In [15]:
site = 'https://pjt3591oo.github.io'
page = ''

while True:
    #딜레이
    time.sleep(1)
    # 기존 출력된거 청소하기
    clear_output( wait=True)
    
    # 페이지 요청
    soup = getSource(site + page)
    # 데이터 수집
    getData(soup)
    # 다음 페이지 존재 확인
    page = getNextPage(soup)

    # 다음 페이지가 없다면 중단한다.
    if page == None :
        print('수집완료')
        break

수집완료
